## 성진님 feature :
'temp', 'pred', 'wind', 'hum', 'day', 'month', 'week', 'sin_time', 'cos_time', 'THI', 'feels_temp', 'CDH', 'hour_mean', 'heat_index'(구간나눔), 'heat_wave(뺴도될 듯)', 'day_hour_mean', 'hour_std', 'last_week_power'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

DATA_PATH = "/content/drive/MyDrive/데이콘 캐글 컴페티션/DACON_23.07_PowerConsumption/"
SEED = 42

In [3]:
import matplotlib.pyplot as plt

In [4]:
train_ft = pd.read_csv(f"{DATA_PATH}train.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test.csv")
building_info = pd.read_csv(f"{DATA_PATH}building_info.csv")

In [5]:
# train_df = train_df[(train_df['건물번호'] >= 69) & (train_df['건물번호'] <= 100)]
# test_df = test_df[(test_df['건물번호'] >= 69) & (test_df['건물번호'] <= 100)]
# building_info = building_info[(building_info['건물번호'] >= 69) & (building_info['건물번호'] <= 100)]

In [6]:
train_ft

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [7]:
train_ft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_date_time  204000 non-null  object 
 1   건물번호           204000 non-null  int64  
 2   일시             204000 non-null  object 
 3   기온(C)          204000 non-null  float64
 4   강수량(mm)        43931 non-null   float64
 5   풍속(m/s)        203981 non-null  float64
 6   습도(%)          203991 non-null  float64
 7   일조(hr)         128818 non-null  float64
 8   일사(MJ/m2)      116087 non-null  float64
 9   전력소비량(kWh)     204000 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 15.6+ MB


In [8]:
test_ft

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77
...,...,...,...,...,...,...,...
16795,100_20220831 19,100,20220831 19,22.5,0.0,0.9,84
16796,100_20220831 20,100,20220831 20,20.7,0.0,0.4,95
16797,100_20220831 21,100,20220831 21,20.2,0.0,0.4,98
16798,100_20220831 22,100,20220831 22,20.1,0.0,1.1,97


In [9]:
test_ft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num_date_time  16800 non-null  object 
 1   건물번호           16800 non-null  int64  
 2   일시             16800 non-null  object 
 3   기온(C)          16800 non-null  float64
 4   강수량(mm)        16800 non-null  float64
 5   풍속(m/s)        16800 non-null  float64
 6   습도(%)          16800 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 918.9+ KB


In [10]:
building_info

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,-,-,-
96,97,호텔및리조트,55144.67,25880.00,-,-,-
97,98,호텔및리조트,53578.62,17373.75,-,-,-
98,99,호텔및리조트,53499.00,40636.00,-,-,-


In [11]:
building_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    object 
 5   ESS저장용량(kWh)  100 non-null    object 
 6   PCS용량(kW)     100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


In [12]:
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].replace('-', 0).astype(float)
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].replace('-', 0).astype(float)
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].replace('-', 0).astype(float)

In [13]:
building_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    float64
 5   ESS저장용량(kWh)  100 non-null    float64
 6   PCS용량(kW)     100 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 5.6+ KB


In [14]:
train_ft = pd.merge(train_ft, building_info, on='건물번호', how='left')
test_ft = pd.merge(test_ft, building_info, on='건물번호', how='left')

In [15]:
# num_date_time, 태양광, ESS, PCS 제거
train_ft = train_ft.drop(columns=['num_date_time', '태양광용량(kW)','ESS저장용량(kWh)', 'PCS용량(kW)'])
test_ft = test_ft.drop(columns=['num_date_time', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)'])

In [16]:
train_ft.columns

Index(['건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전력소비량(kWh)', '건물유형', '연면적(m2)', '냉방면적(m2)'],
      dtype='object')

In [17]:
# 변수 영문명으로 변경
train_cols = ['building_num', 'date_time', 'temp', 'precip', 'wind', 'hum', 'sunshine', 'solar_rad', 'energy_consum', 'building_type', 'floor_area', 'cooling_area']
test_cols = ['building_num', 'date_time', 'temp', 'precip', 'wind', 'hum', 'building_type', 'floor_area', 'cooling_area']

train_ft.columns = train_cols
test_ft.columns = test_cols

# 전력량 맨 앞으로 이동
train_ft = train_ft[['energy_consum','building_num', 'date_time', 'temp', 'precip', 'wind', 'hum', 'sunshine', 'solar_rad', 'building_type', 'floor_area', 'cooling_area']]

In [18]:
train_ft.head()

,energy_consum,building_num,date_time,temp,precip,wind,hum,sunshine,solar_rad,building_type,floor_area,cooling_area
0,1085.28,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,건물기타,110634.0,39570.0
1,1047.36,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,건물기타,110634.0,39570.0
2,974.88,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,건물기타,110634.0,39570.0
3,953.76,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,건물기타,110634.0,39570.0
4,986.40,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,건물기타,110634.0,39570.0


In [19]:
test_ft.head()

,building_num,date_time,temp,precip,wind,hum,building_type,floor_area,cooling_area
0,1,20220825 00,23.5,0.0,2.2,72,건물기타,110634.0,39570.0
1,1,20220825 01,23.0,0.0,0.9,72,건물기타,110634.0,39570.0
2,1,20220825 02,22.7,0.0,1.5,75,건물기타,110634.0,39570.0
3,1,20220825 03,22.1,0.0,1.3,78,건물기타,110634.0,39570.0
4,1,20220825 04,21.8,0.0,1.0,77,건물기타,110634.0,39570.0


In [20]:
# 면적 제거
train_ft = train_ft.drop(columns=['floor_area', 'cooling_area','sunshine','solar_rad'])
test_ft = test_ft.drop(columns=['floor_area', 'cooling_area'])

In [21]:
# 강수랑, 일조, 일사, 풍속 결측치 0.0으로 채우기
train_ft['precip'] = train_ft['precip'].fillna(0.0)
# train_ft['sunshine'] = train_ft['sunshine'].fillna(0.0)
# train_ft['solar_rad'] = train_ft['solar_rad'].fillna(0.0)
train_ft['wind'] = train_ft['wind'].fillna(0.0)

In [22]:
# # 결측치 처리
# mask = train_ft['building_num'] == 9
# train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

# mask = train_ft['building_num'] == 15
# train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

# mask = train_ft['building_num'] == 87
# train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

# mask = train_ft['building_num'] == 90
# train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

# mask = train_ft['building_num'] == 100
# train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

---

#### train data

In [23]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
date = pd.to_datetime(train_ft['date_time'])
train_ft['time'] = date.dt.hour
train_ft['day'] = date.dt.weekday
train_ft['month'] = date.dt.month
train_ft['week'] = date.dt.isocalendar().week
train_ft['day_of_year'] = date.dt.dayofyear

train_ft['sin_time'] = np.sin(2*np.pi*train_ft['time']/24)
train_ft['cos_time'] = np.cos(2*np.pi*train_ft['time']/24)


---
#### test data


In [24]:
date = pd.to_datetime(test_ft['date_time'])
test_ft['time'] = date.dt.hour
test_ft['day'] = date.dt.weekday
test_ft['month'] = date.dt.month
test_ft['week'] = date.dt.isocalendar().week
test_ft['day_of_year'] = date.dt.dayofyear

test_ft['sin_time'] = np.sin(2*np.pi*test_ft['time']/24)
test_ft['cos_time'] = np.cos(2*np.pi*test_ft['time']/24)

In [25]:
# time 칼럼 제거
# test_df = test_df.drop(columns=['time'])

train_ft['week'] = train_ft['week'].astype(int)
test_ft['week'] = test_ft['week'].astype(int)

In [26]:
# 6월 1일 삭제 --> day_of_year == 152
mask_train = train_ft['day_of_year'] != 152
train_ft = train_ft[mask_train].reset_index(drop=True)
train_ft.shape

(201600, 15)

- rolling_avg - 노이즈 제거

In [27]:
train_RM = train_ft[['date_time', 'building_num', 'temp', 'precip', 'wind', 'hum']]
test_RM = train_ft[['date_time', 'building_num', 'temp', 'precip', 'wind', 'hum']]

In [28]:
rolling_mean_ft = pd.concat([train_RM,test_RM], axis=0).reset_index(drop=True)
rolling_mean_ft.shape , rolling_mean_ft.isnull().sum().sum()

((403200, 6), 18)

In [29]:
# 7일치
window_size = 168
rolling_mean_ft['temp_RM'] = rolling_mean_ft.groupby('building_num')['temp'].rolling(window=window_size).mean().reset_index(level=0, drop=True)
rolling_mean_ft['pred_RM'] = rolling_mean_ft.groupby('building_num')['precip'].rolling(window=window_size).mean().reset_index(level=0, drop=True)
rolling_mean_ft['wind_RM'] = rolling_mean_ft.groupby('building_num')['wind'].rolling(window=window_size).mean().reset_index(level=0, drop=True)
rolling_mean_ft['hum_RM'] = rolling_mean_ft.groupby('building_num')['hum'].rolling(window=window_size).mean().reset_index(level=0, drop=True)
rolling_mean_ft

,date_time,building_num,temp,precip,wind,hum,temp_RM,pred_RM,wind_RM,hum_RM
0,20220602 00,1,18.1,0.0,0.4,67.0,NaN,NaN,NaN,NaN
1,20220602 01,1,17.6,0.0,0.8,69.0,NaN,NaN,NaN,NaN
2,20220602 02,1,17.2,0.0,0.7,71.0,NaN,NaN,NaN,NaN
3,20220602 03,1,17.2,0.0,2.2,75.0,NaN,NaN,NaN,NaN
4,20220602 04,1,17.2,0.0,2.0,81.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
403195,20220824 19,100,23.1,0.0,0.9,86.0,25.341667,0.299405,1.026786,86.809524
403196,20220824 20,100,22.4,0.0,1.3,86.0,25.327976,0.299405,1.026190,86.875000
403197,20220824 21,100,21.3,0.0,1.0,92.0,25.321429,0.299405,1.029167,86.892857
403198,20220824 22,100,21.0,0.0,0.3,94.0,25.311905,0.299405,1.023810,86.928571


In [30]:
rolling_mean_ft.iloc[168-1+2016*3] # 잘 반영되었는지 확인

date_time       20220608 23
building_num              4
temp                   20.0
precip                  0.0
wind                    1.7
hum                    73.0
temp_RM           22.194048
pred_RM            0.025595
wind_RM            2.417857
hum_RM            61.541667
Name: 6215, dtype: object

In [31]:
RM_cols = ['temp_RM','pred_RM','wind_RM','hum_RM']

# train_ft에 concat
train_RM = rolling_mean_ft[:201600][RM_cols]
train_ft = pd.concat([train_ft,train_RM],axis=1)

# test_ft에 concat
test_RM = rolling_mean_ft[-16800:][RM_cols].reset_index(drop=True)
test_ft = pd.concat([test_ft,test_RM],axis=1)

train_ft.shape , test_ft.shape # 전력량, 일조량, 일사량

((201600, 19), (16800, 18))

In [32]:
train_ft['day_of_year'].unique()

array([153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204,
       205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,
       218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230,
       231, 232, 233, 234, 235, 236])

In [33]:
# 6월 2일 ~ 6월 8일 마저 삭제
mask_train = train_ft['day_of_year'] > 159
train_ft = train_ft[mask_train].reset_index(drop=True)
train_ft.head()

,energy_consum,building_num,date_time,temp,precip,wind,hum,building_type,time,day,month,week,day_of_year,sin_time,cos_time,temp_RM,pred_RM,wind_RM,hum_RM
0,1042.08,1,20220609 00,19.7,0.0,2.4,77.0,건물기타,0,3,6,23,160,0.000000,1.000000,22.203571,0.025595,2.429762,61.601190
1,933.60,1,20220609 01,19.3,0.0,1.8,80.0,건물기타,1,3,6,23,160,0.258819,0.965926,22.213690,0.025595,2.435714,61.666667
2,871.68,1,20220609 02,19.0,0.0,1.7,85.0,건물기타,2,3,6,23,160,0.500000,0.866025,22.224405,0.025595,2.441667,61.750000
3,850.08,1,20220609 03,18.8,0.0,1.9,87.0,건물기타,3,3,6,23,160,0.707107,0.707107,22.233929,0.025595,2.439881,61.821429
4,888.00,1,20220609 04,18.8,0.0,2.1,86.0,건물기타,4,3,6,23,160,0.866025,0.500000,22.243452,0.025595,2.440476,61.851190


In [34]:
# 결측치 처리
mask = train_ft['building_num'] == 9
train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

mask = train_ft['building_num'] == 15
train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

mask = train_ft['building_num'] == 87
train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

mask = train_ft['building_num'] == 90
train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

mask = train_ft['building_num'] == 100
train_ft.loc[mask, 'hum'] = train_ft['hum'][mask].interpolate()

In [35]:
train_ft.isnull().sum().sum() , test_ft.isnull().sum().sum()

(835, 168)

In [36]:
# 강수랑, 일조, 일사, 풍속 결측치 0.0으로 채우기
train_ft['precip'] = train_ft['precip'].fillna(0.0)
# train_ft['sunshine'] = train_ft['sunshine'].fillna(0.0)
# train_ft['solar_rad'] = train_ft['solar_rad'].fillna(0.0)
train_ft['wind'] = train_ft['wind'].fillna(0.0)

In [37]:
train_ft['THI'] = 1.8*train_ft['temp_RM'] - 0.55*(1-train_ft['hum_RM']/100)*(1.8*train_ft['temp_RM']-26)+32
test_ft['THI'] = 1.8*test_ft['temp_RM'] - 0.55*(1 - test_ft['hum_RM']/100)*(1.8*test_ft['temp_RM'] - 26)+32

In [38]:
# 체감온도 변수 추가
def wet_temp(temp, hum):
    tw = temp*np.arctan(0.151977*(hum + 8.313659)*0.5) + np.arctan(temp + hum) - np.arctan(hum - 1.67633) + 0.00391838*1.5*np.arctan(0.023101*hum) - 4.686035
    return tw

def feels_temp(temp, hum):
    tw = wet_temp(temp, hum)
    ft = -0.2442 + 0.55399*tw + 0.45535*temp - 0.0022*(tw**2) + 0.00278*tw*temp + 3.0
    return ft

In [39]:
train_ft['feels_temp'] = feels_temp(train_ft['temp_RM'], train_ft['hum_RM'])

test_ft['feels_temp'] = feels_temp(test_ft['temp_RM'], test_ft['hum_RM'])

In [40]:
# cdh(cooling degree hour) 변수 추가
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

In [41]:
cdhs = np.array([])
for num in range(1,101,1):
    temp = train_ft[train_ft['building_num'] == num]
    cdh = CDH(temp['temp_RM'].values)
    cdhs = np.concatenate([cdhs, cdh])
train_ft['CDH'] = cdhs

cdhs = np.array([])
for num in range(1,101,1):
    temp = test_ft[test_ft['building_num'] == num]
    cdh = CDH(temp['temp_RM'].values)
    cdhs = np.concatenate([cdhs, cdh])
test_ft['CDH'] = cdhs

In [42]:
# 시간별 전력량 평균 추가
power_hour_mean = train_ft.groupby(['building_num', 'time'])['energy_consum'].mean().reset_index()
train_ft['hour_mean'] = train_ft.apply(lambda x :power_hour_mean.loc[(power_hour_mean.building_num == x['building_num']) & (power_hour_mean.time == x['time']), 'energy_consum'].values[0], axis=1)
test_ft['hour_mean'] = test_ft.apply(lambda x :power_hour_mean.loc[(power_hour_mean.building_num == x['building_num']) & (power_hour_mean.time == x['time']), 'energy_consum'].values[0], axis=1)

In [43]:
# 요일별 시간별 전력량 평균 추가
power_mean = pd.pivot_table(train_ft, values = 'energy_consum', index = ['building_num', 'time', 'day'], aggfunc = np.mean).reset_index()

train_ft['day_hour_mean'] = train_ft.apply(lambda x : power_mean.loc[(power_mean.building_num == x['building_num']) & (power_mean.time == x['time']) & (power_mean.day == x['day']) ,'energy_consum'].values[0], axis = 1)
test_ft['day_hour_mean'] = test_ft.apply(lambda x : power_mean.loc[(power_mean.building_num == x['building_num']) & (power_mean.time == x['time']) & (power_mean.day == x['day']) ,'energy_consum'].values[0], axis = 1)

In [44]:
# 시간별 전력량 표준편차 추
power_hour_std = pd.pivot_table(train_ft, values = 'energy_consum', index = ['building_num', 'time'], aggfunc = np.std).reset_index()

train_ft['hour_std'] = train_ft.apply(lambda x : power_hour_std.loc[(power_hour_std.building_num == x['building_num']) & (power_hour_std.time == x['time']) ,'energy_consum'].values[0], axis = 1)
test_ft['hour_std'] = test_ft.apply(lambda x : power_hour_std.loc[(power_hour_std.building_num == x['building_num']) & (power_hour_std.time == x['time']) ,'energy_consum'].values[0], axis = 1)

- 열지수 구간

In [45]:
# 열지수 단계별로 구분
T = train_ft['temp']*9/5 + 32 # 화씨로 전환
RH = train_ft['hum']
train_ft['heat_index'] = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH
train_ft['heat_index'] = (train_ft['heat_index']-32) * 5/9 # 섭씨로 전환
train_ft.loc[train_ft['heat_index']<32, 'heat_index'] = 0
train_ft.loc[(train_ft['heat_index']>=32) & (train_ft['heat_index']<41), 'heat_index'] = 1
train_ft.loc[(train_ft['heat_index']>=41) & (train_ft['heat_index']<54), 'heat_index'] = 2
train_ft.loc[(train_ft['heat_index']>=54) & (train_ft['heat_index']<66), 'heat_index'] = 3
train_ft.loc[train_ft['heat_index']>=66, 'heat_index'] = 4

T = test_ft['temp']*9/5 + 32 # 화씨로 전환
RH = test_ft['hum']
test_ft['heat_index'] = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH
test_ft['heat_index'] = (test_ft['heat_index']-32) * 5/9 # 섭씨로 전환
test_ft.loc[test_ft['heat_index']<32, 'heat_index'] = 0
test_ft.loc[(test_ft['heat_index']>=32) & (test_ft['heat_index']<41), 'heat_index'] = 1
test_ft.loc[(test_ft['heat_index']>=41) & (test_ft['heat_index']<54), 'heat_index'] = 2
test_ft.loc[(test_ft['heat_index']>=54) & (test_ft['heat_index']<66), 'heat_index'] = 3
test_ft.loc[test_ft['heat_index']>=66, 'heat_index'] = 4

- 폭염 여부

In [46]:
# 폭염 여부 추가
train_ft['heat_wave'] = 0
train_ft.loc[train_ft['temp'] >= 33, 'heat_wave'] = 1

test_ft['heat_wave'] = 0
test_ft.loc[test_ft['temp'] >= 33, 'heat_wave'] = 1

In [47]:
train_ft.columns

Index(['energy_consum', 'building_num', 'date_time', 'temp', 'precip', 'wind',
       'hum', 'building_type', 'time', 'day', 'month', 'week', 'day_of_year',
       'sin_time', 'cos_time', 'temp_RM', 'pred_RM', 'wind_RM', 'hum_RM',
       'THI', 'feels_temp', 'CDH', 'hour_mean', 'day_hour_mean', 'hour_std',
       'heat_index', 'heat_wave'],
      dtype='object')

In [48]:
test_ft.head(10)

,building_num,date_time,temp,precip,wind,hum,building_type,time,day,month,...,wind_RM,hum_RM,THI,feels_temp,CDH,hour_mean,day_hour_mean,hour_std,heat_index,heat_wave
0,1,20220825 00,23.5,0.0,2.2,72,건물기타,0,3,8,...,1.750595,82.601190,76.785354,32.571094,-0.014881,1769.725714,1681.745455,420.573389,0.0,0
1,1,20220825 01,23.0,0.0,0.9,72,건물기타,1,3,8,...,1.744048,82.500000,76.788317,32.580143,-0.020833,1683.827532,1602.458182,415.634605,0.0,0
2,1,20220825 02,22.7,0.0,1.5,75,건물기타,2,3,8,...,1.753571,82.428571,76.792732,32.588300,-0.019048,1564.204675,1478.574545,389.774455,0.0,0
3,1,20220825 03,22.1,0.0,1.3,78,건물기타,3,3,8,...,1.761905,82.309524,76.801356,32.602870,-0.003571,1489.785974,1415.738182,373.002092,0.0,0
4,1,20220825 04,21.8,0.0,1.0,77,건물기타,4,3,8,...,1.773810,82.184524,76.814098,32.620990,0.028571,1497.241558,1423.330909,364.994800,0.0,0
5,1,20220825 05,21.6,0.0,1.6,81,건물기타,5,3,8,...,1.782143,82.077381,76.837541,32.646073,0.082738,1631.174026,1567.985455,379.135642,0.0,0
6,1,20220825 06,21.5,0.0,2.3,84,건물기타,6,3,8,...,1.786905,81.946429,76.856267,32.669203,0.157738,1893.185455,1909.701818,386.636169,0.0,0
7,1,20220825 07,21.7,0.0,1.4,83,건물기타,7,3,8,...,1.797024,81.797619,76.882520,32.699316,0.259524,2259.250909,2420.770909,447.303887,0.0,0
8,1,20220825 08,22.3,0.0,2.2,82,건물기타,8,3,8,...,1.812500,81.613095,76.914194,32.736046,0.394048,2711.183377,2869.789091,439.528549,0.0,0
9,1,20220825 09,22.7,0.0,2.4,78,건물기타,9,3,8,...,1.827976,81.416667,76.953023,32.779127,0.566667,3159.911688,3279.883636,455.826557,0.0,0


In [49]:
train_ft.columns

Index(['energy_consum', 'building_num', 'date_time', 'temp', 'precip', 'wind',
       'hum', 'building_type', 'time', 'day', 'month', 'week', 'day_of_year',
       'sin_time', 'cos_time', 'temp_RM', 'pred_RM', 'wind_RM', 'hum_RM',
       'THI', 'feels_temp', 'CDH', 'hour_mean', 'day_hour_mean', 'hour_std',
       'heat_index', 'heat_wave'],
      dtype='object')

In [50]:
train_ft.fillna(0), test_ft.fillna(0)

(        energy_consum  building_num    date_time  temp  precip  wind   hum  \
 0             1042.08             1  20220609 00  19.7     0.0   2.4  77.0   
 1              933.60             1  20220609 01  19.3     0.0   1.8  80.0   
 2              871.68             1  20220609 02  19.0     0.0   1.7  85.0   
 3              850.08             1  20220609 03  18.8     0.0   1.9  87.0   
 4              888.00             1  20220609 04  18.8     0.0   2.1  86.0   
 ...               ...           ...          ...   ...     ...   ...   ...   
 184795         881.04           100  20220824 19  23.1     0.0   0.9  86.0   
 184796         798.96           100  20220824 20  22.4     0.0   1.3  86.0   
 184797         825.12           100  20220824 21  21.3     0.0   1.0  92.0   
 184798         640.08           100  20220824 22  21.0     0.0   0.3  94.0   
 184799         540.24           100  20220824 23  20.7     0.0   0.1  95.0   
 
        building_type  time  day  ...   wind_RM   

In [51]:
train_ft.to_csv(f'{DATA_PATH}train_feature9.csv', index=False)
test_ft.to_csv(f'{DATA_PATH}test_feature9.csv', index=False)

In [52]:
train_ft.shape

(184800, 27)

In [53]:
test_ft.shape

(16800, 26)

In [54]:
1848/24


77.0